In [70]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import numpy as np

PATH = "C:\\Program Files (x86)\\chromedriver.exe"

In [2]:
def search(driver, position):
    
    # enter position name in search bar
    search = driver.find_element_by_name("q")
    search.send_keys(position)

    # clear loaction name
    location = driver.find_element_by_id('text-input-where')
    location.send_keys(Keys.CONTROL + "a")
    location.send_keys(Keys.DELETE)

    # execute search
    search.send_keys(Keys.RETURN)

In [90]:
def get_job_data(drivers):
    
    # job elements
    jobs = drivers[0].find_elements(By.CLASS_NAME,'result')

    # list for data collection
    data = []

    # iterate over jobs
    for job in jobs:
        time.sleep(np.random.rand() / 5)

        # access individual job container frame
        job.click()
        job_container = drivers[0].find_element(By.ID, "vjs-container-iframe")

        # open job container in seperate chrome driver
        job_link = job_container.get_attribute("src")
        drivers[1].get(job_link)
        time.sleep(np.random.rand() / 5)

        # access job title, company name, location, and description
        title = drivers[1].find_element(By.CLASS_NAME, "jobsearch-JobInfoHeader-title-container").text.split("\n")[0]
        company_location = drivers[1].find_element(By.CLASS_NAME, "icl-u-xs-mt--xs").text.split("\n")
        description = drivers[1].find_element(By.ID, "jobDescriptionText").text.replace("\n", " ")

        # unpack company and location
        company = company_location[0]
        location = company_location[-1]

        # check for salary information
        try:
            salary = drivers[1].find_element(By.CLASS_NAME, "salary-snippet").text
        except:
            salary = "NA"

        # structure data
        data.append({
            "title": title,
            "company": company,
            "loc": location,
            "salary": salary,
            "desc": description
        })
        
    return data

In [91]:
# create drivers, access indeede
drivers = [webdriver.Chrome(PATH) for i in range(2)]
drivers[0].get("https://www.indeed.com/")

# seach for data science, get jobs on page
search(drivers[0], "data science")
data = get_job_data(drivers)

# go to next page, exit popup
drivers[0].find_element(By.CLASS_NAME, "np").click()
drivers[0].find_element(By.CLASS_NAME, "icl-Icon icl-Icon--sm").click()

# close drivers
# [driver.close() for driver in drivers]
print(f"Found {len(data)} jobs")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".np"}
  (Session info: chrome=93.0.4577.82)
